<a href="https://colab.research.google.com/github/choheeee22/Section2_Project/blob/main/Section2__%EC%88%98%EC%9A%94%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

목차


1. 프로젝트 목적 소개

2. 프로젝트 진행 절차 (과정, 가설 및 평가지표) 

3. 데이터 소개 

4. 사용한 머신 러닝 기법 

5. 결론







1. 프로젝트 목적 소개

배경: 공장의 라인 운영과 긴 원부자재 리드타임 고려해, 바이어 오더 수주 전 우리 회사의 부담으로 미리 오더를 예상해 자재 선발주 진행. 라인 블랭크 비용 및 바이어와의 관계 유지 감안 시 이와 같은 선발주는 피할 수 없는 상황.

문제 정의: 실제 오더 수주가 되지 않을 경우, 이에 소요된 원/부자재, 공장 운영 비용이 모두 회사 비용으로 책정 

현재 실행 방법: 이전 분기, 년도의 수주 받은 오더를 참고하여 스타일/색상등을 경험치로 예상 원부자재 발주 진행

제안 방법: 현재 예측 방법(수주 받은 오더 분석)에 바이어 세일즈 정보 및 미국 경제 지표에 대한 분석을 추가해 예측의 정확도를 높여 비용을 최소화 

2. 프로젝트 진행 절차 (과정, 가설 및 평가지표) 

진행 절차 소개 

바이어 sales / 국가 경제지표 data를 시간에 따라 세 그룹으로 분류 (학습/검증/테스트).


학습 데이터를 통해 모델을 익히고, 검증 데이터로 어떤 모델이 우위를 평가 후, 최종 테스트 데이터 점수를 통해 미래의 세일즈 데이터를 얼마나 잘 예측하는지 평가


가설 소개 


가설1 boosting 기법이 성능이 제일 우수 할 것이다 (점수는 1점 만점에 0.8 이상 기대)


가설2 매 년 시기마다 비슷한 세일즈를 보일 것이다.

평가 지표

MAE (Mean Absolute Error) - 예상치와 실제값의 차이 절대값의 평균 

선정 이유:공휴일/ 피크시즌의 세일즈가 폭발 할 경우 이상치로 잡지 않아 객관적인 평가가 가능하므로 선정 

R-Squared - 데이터에 대한 모델의 설명정도를 나타냄 (0~1)

선정 이유: 각 모델이 데이터를 얼마나 잘 설명하는 지 파악 위해 선정

3. 데이터 소개 

바이어: Walmart


데이터 기간: 2010-02-05 to 2012-11-01


변수: Store, Dept, 공휴일 유무(Super Bowl, Labor Day, Thanksgiving, Christmas), 기온, Markdown, CPI, 실업율, 유가 


In [ ]:
#데이터 불러오기 
from google.colab import files
uploaded= files.upload()

Saving features.csv to features.csv
Saving stores.csv to stores.csv
Saving train.csv to train.csv


In [ ]:
import pandas as pd 
features=pd.read_csv("features.csv")
stores=pd.read_csv("stores.csv")
train=pd.read_csv("train.csv")

In [ ]:
#features 중 2012-11-01 (train 상 2012/10/26 이후 데이터 - over)/ features 에서 10/26 초과 데이터제거 -- features_2로 저장  (rows-6435)
features.Date.sort_values(ascending=False)
over=features[pd.to_datetime(features.Date)>pd.to_datetime("2012-10-26")]
over
features_2=features[~features['Date'].isin(over.Date)]
features_2.Date.sort_values(ascending=True)
features_2